In [3]:
import pandas as pd
import numpy as np
import os
from tensorflow.keras.utils import load_img
import matplotlib.pyplot as pt

In [1]:
input_path = []
label = []
for folder_name in os.listdir('PetImages'):
    for folder_path in os.listdir('PetImages/'+folder_name):
        if folder_name == 'Cat':
            label.append('00')
        else:
            label.append('11')
        input_path.append(os.path.join('PetImages', folder_name, folder_path))

In [4]:
df = pd.DataFrame()
df['images'] = input_path
df['label'] = label
df = df.sample(frac=1).reset_index(drop=True)
df.head(5)

,images,label
0,PetImages\Cat\2110.jpg,00
1,PetImages\Dog\5607.jpg,11
2,PetImages\Dog\10113.jpg,11
3,PetImages\Dog\6073.jpg,11
4,PetImages\Dog\9104.jpg,11


In [8]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size = 0.2, random_state = 0)

from keras.preprocessing.image import ImageDataGenerator
training_generator = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 40, 
    shear_range = 0.2,
    horizontal_flip = True,
    fill_mode = 'nearest'
)

validation_generator = ImageDataGenerator(rescale = 1./255)
training_iterator = training_generator.flow_from_dataframe(
    train, 
    x_col = 'images',
    y_col = 'label',
    target_size = (155,155),
    batch_size=512,
    class_mode = 'binary'
)

validation_iterator = validation_generator.flow_from_dataframe(
   test,
   x_col = 'images',
   y_col = 'label',
   target_size=(155,155),
   batch_size = 512,
   class_mode = 'binary'

)

Found 20001 validated image filenames belonging to 2 classes.
Found 4999 validated image filenames belonging to 2 classes.


c:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\preprocessing\image.py:989: UserWarning: Found 2 invalid image filename(s) in x_col="images". These filename(s) will be ignored.
  warnings.warn('Found {} invalid image filename(s) in x_col="{}". '


In [9]:
from keras import Sequential
from keras.layers import Conv2D, Flatten, MaxPool2D, Dense

model = Sequential()
model.add(Conv2D(16, (3,3), activation = 'relu', input_shape = (155,155,3)))
model.add(MaxPool2D(2,2))
model.add(Conv2D(32, (3,3), activation = 'relu'))
model.add(MaxPool2D(2,2))
model.add(Conv2D(64, (3,3), activation = 'relu'))
model.add(MaxPool2D(2,2))
model.add(Flatten())
model.add(Dense(512, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))


In [10]:
model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [11]:
prediction = model.fit(training_iterator, epochs = 5, validation_data = validation_iterator)

UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x000001DB5C53F100>